In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
# https://lightgbm.readthedocs.io/en/latest/Python-API.html#lightgbm.LGBMModel
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, train_test_split

from hyperopt import fmin, tpe, hp
from sklearn.cross_validation import cross_val_score
from bayes_opt import BayesianOptimization

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# SVR optimization methods

In [2]:
def bayesian_optimization_svr(X, y, cv=6, max_iter_svr=100, max_iter_opt=15):
    svr_opt = BayesianOptimization(
        lambda epsilon: cross_val_score(
            LinearSVR(C=100, epsilon=epsilon, max_iter=max_iter_svr),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        {'epsilon': (0.0, 5.0)},
        verbose=0
    )
    
    svr_opt.init(10)
    svr_opt.maximize(n_iter=max_iter_opt)
    
    return svr_opt.res['max']['max_params']['epsilon']

def hyperopt_optimization_svr(X, y, cv=6, max_iter_svr=100, max_iter_opt=15):
    space = hp.choice('regressor_type', [
        {
            'type': 'svr',
            'epsilon': hp.uniform('svr_epsilon', 0.0, 5.0)
        }
    ])

    best = fmin(
        fn=lambda args: cross_val_score(
            LinearSVR(C=100, epsilon=args['epsilon'], max_iter=max_iter_svr),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        space=space,
        algo=tpe.suggest,
        max_evals=max_iter_opt
    )

    return best['svr_epsilon']

def evaulate_best_svr_argument(X, y, eps_vals: list, cv=6):
    scores = []
    for e in eps_vals:
        scores.append(cross_val_score(
            LinearSVR(C=100, epsilon=e, max_iter=200),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean())
        
    return eps_vals[np.argmax(scores)]

# RF regressor optimization code

In [3]:
# hyperopt for lightgbm shows terrible results
def hyperopt_optimization_lightgbm(X, y, cv=6, max_iter_opt=15):
    space = hp.choice('regressor_type', [
        {
            'type': 'lightgbm',
            'feature_fraction': hp.uniform('feature_fraction', 0.05, 0.95),
            'bagging_fraction': hp.uniform('bagging_fraction', 0.05, 0.95),
            'bagging_freq': hp.uniform('bagging_freq', 1, 50),
            'n_estimators': hp.uniform('n_estimators', 5, 50),
            #'max_bin': hp.uniform('max_bin', )
        }
    ])

    best = fmin(
        fn=lambda args: cross_val_score(
            LGBMRegressor(
                boosting_type='rf', 
                feature_fraction=args['feature_fraction'], 
                bagging_freq=int(args['bagging_freq']), 
                bagging_fraction=args['bagging_fraction'],
                n_estimators=int(args['n_estimators'])
            ),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        space=space,
        algo=tpe.suggest,
        max_evals=max_iter_opt
    )
    
    return best

def bayesian_optimization_lightgbm(X, y, cv=6, max_iter_opt=15):
    svr_opt = BayesianOptimization(
        lambda feature_fraction, bagging_freq, bagging_fraction, n_estimators: cross_val_score(
            LGBMRegressor(
                boosting_type='rf', 
                feature_fraction=feature_fraction, 
                bagging_freq=int(bagging_freq), 
                bagging_fraction=bagging_fraction,
                n_estimators=int(n_estimators)
            ),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        {'feature_fraction': (0.05, 0.95),
         'bagging_fraction': (0.05, 0.95),
         'bagging_freq': (1, 50),
         'n_estimators': (5, 50) },
        verbose=0
    )
    
    svr_opt.init(10)
    svr_opt.maximize(n_iter=max_iter_opt)
    
    return svr_opt.res['max']['max_params']#['C']

# Facebook Comment Volume Dataset

In [4]:
df = pd.read_csv('datasets/facebook_comments.csv', index_col=0)

In [5]:
print('dataset size: {}'.format(df.shape))

dataset size: (40949, 54)


In [6]:
# encoding categorical variables
# H Local - category
# Post Promotion Status - category
# Base Time - time variable
# Page Category - category

H_Local = OneHotEncoder().fit_transform(df['H Local'].values.reshape(-1, 1)).todense()
Post_Promotion_Status = OneHotEncoder().fit_transform(df['Post Promotion Status'].values.reshape(-1, 1)).todense()
Base_Time = OneHotEncoder().fit_transform(df['Base Time'].values.reshape(-1, 1)).todense()
Page_Category = OneHotEncoder().fit_transform(df['Page Category'].values.reshape(-1, 1)).todense()

In [7]:
y = df.Target.values.reshape(-1, 1)
X = df.drop(['H Local', 'Post Promotion Status', 'Base Time', 'Page Category', 'Target'], axis=1).values
X = np.hstack([X, H_Local, Post_Promotion_Status, Base_Time, Page_Category])

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [9]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (40949, 228)


## Training plain model

### * Estimating SVR penalty

In [10]:
X_train, y_train = shuffle(X_train, y_train)

In [11]:
eps_hyperopt = hyperopt_optimization_svr(X_train, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)
eps_bayesian = bayesian_optimization_svr(X_train, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)

In [12]:
eps_opt = evaulate_best_svr_argument(X_train, y_train, [eps_hyperopt, eps_bayesian], cv=6)

### * Estimating LightGBM params

In [14]:
params_opt = bayesian_optimization_lightgbm(X_train, y_train, cv=6, max_iter_opt=50)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-3.39073783]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)


In [15]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

### * Evaluate with CV

In [16]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=100, epsilon=eps_opt, max_iter=200),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

In [17]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_train, y_train):
        X_crossval_train, X_crossval_test = X_train[train_ix], X_train[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        # here must be sume sort of optimization
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))

In [18]:
print('for pure data (averate scores):')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for pure data (averate scores):
linear, MSE: 795.2162610416704
svr, MSE: 1728.8641361699947
forest, MSE: 772.6272131333926


In [19]:
print('for pure data (validation scores):')
for name, model in models.items():
    model.fit(X_train, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val), y_val.squeeze())))

for pure data (validation scores):
linear, MSE: 914.2573871292184
svr, MSE: 275536.7747979826
forest, MSE: 977.3517320750891


### Training model with PCA-processed data

In [20]:
# number of components is fixed to 10
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)

In [21]:
X_pca, y_train = shuffle(X_pca, y_train)

### * Estimating SVR penalty

In [22]:
C_hyperopt = hyperopt_optimization_svr(X_pca, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)
C_bayesian = bayesian_optimization_svr(X_pca, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)

In [23]:
C_opt = evaulate_best_svr_argument(X_pca, y_train, [C_hyperopt, C_bayesian], cv=6)

### * Estimating LightGBM params

In [24]:
params_opt = bayesian_optimization_lightgbm(X_pca, y_train, cv=6, max_iter_opt=50)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.90489854]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.70217373]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)


In [25]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

In [26]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=C_opt),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

### * Evaluate with CV

In [27]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_pca, y_train):
        X_crossval_train, X_crossval_test = X_pca[train_ix], X_pca[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))
        

In [28]:
print('for decorrelated data using PCA (averate scores)')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using PCA (averate scores)
linear, MSE: 907.6697363284552
svr, MSE: 185357.7382693561
forest, MSE: 910.0725575075838


In [29]:
print('for decorrelated data using PCA (validation scores):')
for name, model in models.items():
    model.fit(X_train, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val), y_val.squeeze())))

for decorrelated data using PCA (validation scores):
linear, MSE: 1464.9637832200358
svr, MSE: 95049.85683415309
forest, MSE: 1473.1972658494396


## Parkinsons Telemonitoring Data Set

In [31]:
df = pd.read_csv('datasets/parkinsons_updrs.data')

In [32]:
print('dataset size: {}'.format(df.shape))

dataset size: (5875, 22)


In [33]:
subject = df['subject#'].unique().tolist()
subject_binary = np.zeros((df.shape[0], len(subject)))
for k, i in df.iterrows():
    subject_binary[k, int(i['subject#']) - 1] = 1

In [34]:
# replacing categorical features with binary values
y = df.total_UPDRS.values
X = df.drop(['motor_UPDRS', 'total_UPDRS', 'subject#'], axis=1).values
X = np.concatenate([subject_binary, X], axis=1)

In [35]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [36]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (5875, 61)


## Training plain model

### * Estimating SVR penalty

In [37]:
X_train, y_train = shuffle(X_train, y_train)

In [38]:
C_hyperopt = hyperopt_optimization_svr(X_train, y_train, cv=6, max_iter_svr=1000, max_iter_opt=15)
C_bayesian = bayesian_optimization_svr(X_train, y_train, cv=6, max_iter_svr=1000, max_iter_opt=15)

In [39]:
C_opt = evaulate_best_svr_argument(X_train, y_train, [C_hyperopt, C_bayesian], cv=6)

### * Estimating LightGBM params

In [40]:
params_opt = bayesian_optimization_lightgbm(X_train, y_train, cv=6, max_iter_opt=50)

In [41]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

### * Evaluate with CV

In [42]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=C_opt, max_iter=1000),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

In [43]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_train, y_train):
        X_crossval_train, X_crossval_test = X_train[train_ix], X_train[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        # here must be sume sort of optimization
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))

In [44]:
print('for pure data (averate scores):')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for pure data (averate scores):
linear, MSE: 6.813584776542285
svr, MSE: 17.60012254051052
forest, MSE: 8.009305240274019


In [45]:
print('for pure data (validation scores):')
for name, model in models.items():
    model.fit(X_train, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val), y_val.squeeze())))

for pure data (validation scores):
linear, MSE: 6.098093058357888
svr, MSE: 9.046091209340968
forest, MSE: 7.484697650054532


### Training model with PCA-processed data

In [46]:
# number of components is fixed to 10
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)

In [47]:
X_pca, y_train = shuffle(X_pca, y_train)

### * Estimating SVR penalty

In [48]:
C_hyperopt = hyperopt_optimization_svr(X_pca, y_train, cv=6, max_iter_svr=1000, max_iter_opt=15)
C_bayesian = bayesian_optimization_svr(X_pca, y_train, cv=6, max_iter_svr=1000, max_iter_opt=15)

In [49]:
C_opt = evaulate_best_svr_argument(X_pca, y_train, [C_hyperopt, C_bayesian], cv=6)

### * Estimating LightGBM params

In [50]:
params_opt = bayesian_optimization_lightgbm(X_pca, y_train, cv=6, max_iter_opt=50)

In [51]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

In [52]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=C_opt),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

### * Evaluate with CV

In [53]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_pca, y_train):
        X_crossval_train, X_crossval_test = X_pca[train_ix], X_pca[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))
        

In [54]:
print('for decorrelated data using PCA (averate scores)')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using PCA (averate scores)
linear, MSE: 83.11222503906008
svr, MSE: 94.73598804808401
forest, MSE: 8.420770416791695


In [55]:
print('for decorrelated data using PCA (validation scores):')
for name, model in models.items():
    model.fit(X_train, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val), y_val.squeeze())))

for decorrelated data using PCA (validation scores):
linear, MSE: 121.4606121530821
svr, MSE: 125.52757627239173
forest, MSE: 120.1179463432551


## Energy efficiency Data Set

In [56]:
df = pd.read_excel('./datasets/ENB2012_data.xlsx')

In [57]:
print('dataset size: {}'.format(df.shape))

dataset size: (768, 10)


In [58]:
subject = df['X6'].unique().tolist()
subject_map = dict(zip(subject, range(len(subject))))
subject_binary = np.zeros((df.shape[0], len(subject)))
for k, i in df.iterrows():
    subject_binary[k, subject_map[i['X6']]] = 1

In [59]:
# replacing categorical features with binary values
y = df.Y1.values
X = df.drop(['Y1', 'Y2', 'X6'], axis=1).values
X = np.concatenate([subject_binary, X], axis=1)

In [60]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [61]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (768, 11)


## Training plain model

### * Estimating SVR penalty

In [62]:
X_train, y_train = shuffle(X_train, y_train)

In [63]:
C_hyperopt = hyperopt_optimization_svr(X_train, y_train, cv=6, max_iter_svr=1000, max_iter_opt=15)
C_bayesian = bayesian_optimization_svr(X_train, y_train, cv=6, max_iter_svr=1000, max_iter_opt=15)

In [64]:
C_opt = evaulate_best_svr_argument(X_train, y_train, [C_hyperopt, C_bayesian], cv=6)

### * Estimating LightGBM params

In [65]:
params_opt = bayesian_optimization_lightgbm(X_train, y_train, cv=6, max_iter_opt=50)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-16.16280141]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 29, 'nit': 1, 'warnflag': 2}
  " state: %s" % convergence_dict)


In [66]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

### * Evaluate with CV

In [67]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=C_opt, max_iter=1000),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

In [68]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_train, y_train):
        X_crossval_train, X_crossval_test = X_train[train_ix], X_train[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        # here must be sume sort of optimization
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))

In [69]:
print('for pure data (averate scores):')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for pure data (averate scores):
linear, MSE: 8.986799494934631
svr, MSE: 32.901727228720524
forest, MSE: 1.0745437484051013


In [70]:
print('for pure data (validation scores):')
for name, model in models.items():
    model.fit(X_train, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val), y_val.squeeze())))

for pure data (validation scores):
linear, MSE: 7.748646909011892
svr, MSE: 17.127049376679032
forest, MSE: 1.3444327311825048


### Training model with PCA-processed data

In [71]:
# number of components is fixed to 4
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)

### * Estimating SVR penalty

In [72]:
X_pca, y_train = shuffle(X_pca, y_train)

In [73]:
C_hyperopt = hyperopt_optimization_svr(X_pca, y_train, cv=6, max_iter_svr=1000, max_iter_opt=15)
C_bayesian = bayesian_optimization_svr(X_pca, y_train, cv=6, max_iter_svr=1000, max_iter_opt=15)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.583638e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)


In [74]:
C_opt = evaulate_best_svr_argument(X_pca, y_train, [C_hyperopt, C_bayesian], cv=6)

### * Estimating LightGBM params

In [75]:
params_opt = bayesian_optimization_lightgbm(X_pca, y_train, cv=6, max_iter_opt=50)

In [76]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

In [77]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=C_opt),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

### * Evaluate with CV

In [78]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_pca, y_train):
        X_crossval_train, X_crossval_test = X_pca[train_ix], X_pca[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))
        

In [79]:
print('for decorrelated data using PCA (averate scores)')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using PCA (averate scores)
linear, MSE: 21.11891304611659
svr, MSE: 59.04677932262165
forest, MSE: 6.704298941195557


In [80]:
print('for decorrelated data using PCA (validation scores):')
for name, model in models.items():
    model.fit(X_train, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val), y_val.squeeze())))

for decorrelated data using PCA (validation scores):
linear, MSE: 103.87685186956993
svr, MSE: 152.16182537274634
forest, MSE: 107.50667933196301
